In [1]:
# Para importar reviews en modo memory-mapping
from datasets import load_dataset
# Para importar business
import polars as pl

In [2]:
# Importamos reviews
reviews = load_dataset('csv', data_files='../../data/yelp/review-001.csv', split='train')

Found cached dataset csv (C:/Users/camat/.cache/huggingface/datasets/csv/default-97eb5bfe6d31b1ab/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


In [3]:
# Importamos business
business = pl.read_parquet('../../data/yelp/production/business.parquet')

In [4]:
# Creamos una lista del new_id y los old_id para los business
company_index = business['company_index'].to_list()
business_id = business['business_id'].to_list()

In [5]:
# Retorna el nuevo id segun el old id. Si el old id no existe entonces devuelve None.
def get_company_index(example):
    old_ids = example['business_id']
    output = []
    for old_id in old_ids:
        if old_id in business_id:
            output.append(company_index[business_id.index(old_id)])
        else:
            output.append(None)
    return {'company_index': output}

In [6]:
# Creamos una columna llamada company_index que contendra el indice de la empresa nuevo asociado en business
result = reviews.map(get_company_index, batch_size=1000, batched=True)

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-97eb5bfe6d31b1ab\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-d4fbc206b668ec32.arrow


In [7]:
# Toda aquel review cuyo company_id antiguo no existe se lo filtra.
result = result.filter(lambda rows: rows['company_index'] is not None)

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-97eb5bfe6d31b1ab\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-cd0117451afdff17.arrow


In [8]:
# Creamos una lista con los usuarios unicos
x = result.unique('user_id')

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-97eb5bfe6d31b1ab\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-efe6e0cc41aec0f8.arrow


In [9]:
# Numero de usuarios unicos
len(x)

1987648

In [10]:
# Creamos un diccionario que mapea old con new ids
map_new_old_ids = {old:int(new + 1) for new, old in enumerate(x)}

In [11]:
# Ejemplo
example = result[0]['user_id']
print('OLD ID:', example)
print('NEW ID:', map_new_old_ids[example])

OLD ID: mh_-eMZ6K5RLWhZyISBhwA
NEW ID: 1


In [12]:
# Creamos la nueva columna llamada user_index
result = result.map(lambda rows: {'user_index': map_new_old_ids[rows['user_id']]})

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-97eb5bfe6d31b1ab\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-5722683e5d7d1516.arrow


In [13]:
# Almacenamos la data
result.to_parquet('../../data/yelp/production/reviews.parquet')

Creating parquet from Arrow format:   0%|          | 0/6989 [00:00<?, ?ba/s]

5037398697